## RDD Transformation and Actions Notebook

Try this notebook on databricks.

In [0]:
spark

Out[10]:

SparkSession - hive 
 
 
 SparkContext 

 Spark UI 

 
 Version 
 v3.1.0 
 Master 
 local[8] 
 AppName 
 Databricks Shell

In [0]:
# spark session vs context
type(spark), type(sc)

Out[14]: (pyspark.sql.session.SparkSession, PythonShellImpl.RemoteContext)

In [0]:
# sc and spark are already initiated by Databricks
spark.sparkContext, sc, sc is spark.sparkContext

Out[16]: (<SparkContext master=local[8] appName=Databricks Shell>,
 <SparkContext master=local[8] appName=Databricks Shell>,
 True)

In [0]:
rdd = sc.parallelize(range(20))

In [0]:
rdd

Out[20]: PythonRDD[13] at RDD at PythonRDD.scala:58

In [0]:
rdd.getNumPartitions() 

Out[22]: 8

In [0]:
rdd.glom().collect()

Out[21]: [[0, 1],
 [2, 3, 4],
 [5, 6],
 [7, 8, 9],
 [10, 11],
 [12, 13, 14],
 [15, 16],
 [17, 18, 19]]

In [0]:
# narrow transformation
# Narrow transformation does not require 
# shuffle or reorganizing data between partitions
filteredRDD = rdd.filter(lambda x: x > 10)

In [0]:
print(filteredRDD.toDebugString()) 

b'(8) PythonRDD[15] at RDD at PythonRDD.scala:58 []\n | ParallelCollectionRDD[12] at readRDDFromInputStream at PythonRDD.scala:413 []'

In [0]:
filteredRDD.collect()

Out[25]: [11, 12, 13, 14, 15, 16, 17, 18, 19]

In [0]:
# wide transformation
# wide dependencies (namely wide transformations) such as groupByKey, reduceByKey, etc. 
# Within these dependencies, the data needed 
# for the processing can be located in several partitions 
# of the parent RDD that need to be combined.
groupedRDD = filteredRDD.groupBy(lambda x: x % 2) 

In [0]:
groupedRDD.toDebugString()

Out[28]: b'(8) PythonRDD[20] at RDD at PythonRDD.scala:58 []\n | MapPartitionsRDD[19] at mapPartitions at PythonRDD.scala:183 []\n | ShuffledRDD[18] at partitionBy at NativeMethodAccessorImpl.java:0 []\n +-(8) PairwiseRDD[17] at groupBy at <command-3405833377265717>:6 []\n | PythonRDD[16] at groupBy at <command-3405833377265717>:6 []\n | ParallelCollectionRDD[12] at readRDDFromInputStream at PythonRDD.scala:413 []'

In [0]:
groupedRDD.collect()

Out[29]: [(0, <pyspark.resultiterable.ResultIterable at 0x7ffa623833d0>),
 (1, <pyspark.resultiterable.ResultIterable at 0x7ffa62383850>)]

In [0]:
# The actions are not lazy — they will actually trigger the data processing
filteredRDD.reduce(lambda a, b: a + b)

Out[31]: 135